In [1]:
import numpy as np
import pandas as pd
from data_extraction import *
from resp_signal_extraction import *
from rr_extration import *
from sklearn.preprocessing import MinMaxScaler
import re
import tensorflow as tf
import pickle as pkl
from tf_model import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber

In [2]:
with open('output','rb') as f:
    output_data = pkl.load(f)

with open('input','rb') as f:
    input_data = pkl.load(f)

input_data = input_data.reshape(input_data.shape[0],input_data.shape[-1],input_data.shape[1])
annotation = pd.read_pickle('/media/hticpose/drive1/charan/BR_Uncertainty/DL_BASED_METHOD/annotation.pkl')
reference_rr = (annotation['Reference_RR'].values).reshape(-1,1)

tensor_input = tf.convert_to_tensor(input_data)
tensor_output = tf.convert_to_tensor(output_data)
tensor_ref_rr = tf.convert_to_tensor(reference_rr)
training_ids = annotation['patient_id'] < 13

x_train_data = tensor_input[tf.convert_to_tensor(training_ids.values)]
x_test_data = tensor_input[tf.convert_to_tensor(~(training_ids.values))]
x_train_ref_rr = tensor_ref_rr[tf.convert_to_tensor(training_ids.values)]
x_test_ref_rr = tensor_ref_rr[tf.convert_to_tensor(~(training_ids.values))]

y_train_data = tensor_output[tf.convert_to_tensor(training_ids.values)]
y_test_data = tensor_output[tf.convert_to_tensor(~(training_ids.values))]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data , y_train_data))
train_dataset = train_dataset.shuffle(len(x_train_data)).batch(128)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data , y_test_data))
test_dataset = test_dataset.batch(128)

In [9]:
y_test_data.shape

TensorShape([814, 128])

In [12]:
model_input_shape = (128,3)
model  = BRUnet(model_input_shape)
optimizer = Adam(learning_rate = 0.005)
loss_fn = Huber()
model(tf.ones((128,128,3)))
model.load_weights('/media/hticpose/drive1/charan/BR_Uncertainty/DL_BASED_METHOD/best_model.h5')

test_loss_list = []
final_output = tf.convert_to_tensor([])

for step , (x_batch_test,y_batch_test) in enumerate(test_dataset):
    output = model(x_batch_test)
    test_loss = loss_fn(y_batch_test , output)
    print(tf.shape(output))
    '''
    if step == 0:
        final_output = output
    else:
        tf.concat([final_output , output] , axis = 0)
    test_loss_list.append(test_loss)
    '''

tf.Tensor([128 128   1], shape=(3,), dtype=int32)
tf.Tensor([128 128   1], shape=(3,), dtype=int32)
tf.Tensor([128 128   1], shape=(3,), dtype=int32)
tf.Tensor([128 128   1], shape=(3,), dtype=int32)
tf.Tensor([128 128   1], shape=(3,), dtype=int32)
tf.Tensor([128 128   1], shape=(3,), dtype=int32)
tf.Tensor([ 46 128   1], shape=(3,), dtype=int32)


[<tf.Tensor: shape=(), dtype=float32, numpy=0.018694442>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.015124833>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.016113013>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.01993818>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.017204897>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.014655525>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011357721>]